In [1]:
import pandas as pd
import jieba
import jieba.analyse
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

pd.set_option('display.max_colwidth', None)#setting the maximize string show

In [2]:
df = pd.read_csv("KCC Data/NewsCisDepAll_Seg-35325.csv", encoding='utf-8-sig') 

In [3]:
df.head(2)

,編號,質詢單位,詳細,逐字稿
0,15601,陳美雅,2-1-1040504-2-陳美雅-1,陳議員美雅：本席的市政總質詢想針對高雄市民關心的許多議題就教市府團隊。首先本席要請教市長，高雄市目前對於大陸之間的兩岸關係，市長的態度如何，認為我們跟大陸之間的交流，你是支持還是反對？請市長表達你的立場。主席（康議長裕成）：市長，請回答。陳市長菊：高雄市是一個開放的城市，高雄市跟中國大陸之間，我們一向都是鼓勵交流。同時我個人為了2009年世界運動會、2013年亞太城市高峰會議，我都曾經到中國大陸相關的城市邀請他們來參與。我想做為一個開放的城市，我們都是…。陳議員美雅：所以市長你認為台灣目前應該維持什麼樣的政治立場，你目前的政治立場可不可以更簡單一點？陳市長菊：台灣現階段就是維持現狀。陳議員美雅：我們的現狀可不可以請市長用更白話一點告訴市民？陳市長菊：這跟市政無關，我想今天不討論政治議題。陳議員美雅：本席現在要了解的，是要跟高雄市民講，如果剛剛市長表達的立場是未來也希望增加兩岸之間的交流…。陳市長菊：我同意。陳議員美雅：那麼市長是否可以更坦然的面對這個問題，讓兩岸之間的交流和平發展？陳市長菊：當然。陳議員美雅：所以你認為目前台灣跟大陸之間朝著所謂的和平發展，是指九二共識你也認同的意思嗎？陳市長菊：兩岸之間朝著更進步的的發展，我想九二共識有不同的解讀，這個部分在台灣社會有不同的看法…。陳議員美雅：市長你的解讀呢？陳市長菊：我想這跟今天的市政無關。謝謝。陳議員美雅：這跟未來高雄市政，大陸未來如何跟高雄市之間做的觀光文化交流，甚至未來有沒有任何經貿交流有關，市長你好像一直迴避這個問題。陳市長菊：我們都贊成跟中國大陸之間不斷的開放交流，藉由民間更多的交流，異中求同，然後爭取更多彼此的認知及認識，這樣兩岸之間才能更趨和緩。謝謝。陳議員美雅：謝謝市長，市長剛剛的意思也尊重所謂的九二共識。陳市長菊：沒有，那是你的解讀，我不是這樣解讀的。陳議員美雅：市長，你的解讀讓你更白話的說一句話，你就用一句話表達你的立場，我的市政質詢時間非常寶貴。陳市長菊：我的答覆也是很寶貴，時間是共同的。陳議員美雅：所以你的立場到底是什麼呢？陳市長菊：我的立場很清楚，兩岸之間異中求同，兩岸之間有很多歷史發展的過程不一樣，兩岸之間應該要更多的往來、互相尊重、異中求同。謝謝。陳議員美雅：互相尊重嘛！謝謝你認同所謂的互相尊重這一點。我想高雄不要再有所謂的意識形態，議員在高雄市議會要為高雄市、為台灣表達一個立場，我們希望讓高雄市民能夠生活得更好，不要總是赤字困難。我們希望能創造高雄跟兩岸之間，甚至各國之間是友好的環境，讓大家樂於來高雄市投資，我們就是希望讓高雄市民未來有更多的就業機會，未來高雄市能自創更多的自有財源。這是市長未來應該要為高雄市民奮鬥努力的。
1,15602,陳美雅,2-1-1040504-2-陳美雅-10,陳議員美雅：請尊重本席的時間。市長，這就是我們的官員，新的局長也要來和議員嘔，難怪副市長做錯誤的示範，各位高雄市民，民進黨完全執政之下，居然就是這樣蠻橫的狀態，對於議員的質詢不針對問題回答，還要用回嗆的方式，甚至沒有請他回答時，他也要在台下用叫囂的方式。本席再請教市長，45億6


In [4]:
df['質詢單位'].unique()

array(['陳美雅', '李喬如', '簡煥宗', '蔡金晏', '國民黨團', '民進黨團'], dtype=object)

In [5]:
ly = df[df.質詢單位 == "國民黨團"]

In [6]:
doc_names = range(len(ly)) #有幾篇文章
doc_names

range(0, 39)

In [7]:
text_list = list(ly['逐字稿'])
ly_docs = pd.DataFrame(columns=['jieba_results'])
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

In [8]:
words = jieba.lcut(text_list[1], cut_all = False)
print(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built successfully.


['李議員眉蓁', '：', '延續', '剛剛', '的', '問題', '，', '現在', '高銀', '董事', '長', '也', '進來', '了', '，', '剛剛', '的', '問題', '不', '知道', '你', '有', '沒', '有', '聽', '到', '，', '最後我們', '再詢', '問高銀', '董事', '長', '，', '請問', '你', '如果', '覺得', '高銀是', '清白', '的', '話', '，', '是否', '願意來', '議會', '專案', '報告', '。', '請', '董事', '長', '回答', '。', '主席', '（', '康議', '長', '裕', '成', '）', '：', '請', '董事', '長', '回答', '。', '高雄', '銀行', '張', '董事', '長', '雲鵬', '：', '跟', '李議員', '報告', '，', '我', '非常', '願意', '到', '高雄市', '議會來', '報告', '。', '雖然', '我', '今年', '6', '月', '才', '到任', '，', '但是', '過去', '的', '事情', '我應', '該', '要', '概括', '承受', '。', '謝謝', '。', '李議員眉蓁', '：', '謝謝', '董事', '長', '，', '請', '坐', '。', '如果', '你', '認為', '你', '是', '清白', '，', '你', '也', '非常', '願意來', '議會', '報告', '的', '話', '，', '希望', '你', '也', '跟', '議會', '提出', '你', '想', '澄清', '這件', '事情', '，', '讓', '我們', '議會', '的', '議員', '不要', '為', '了', '想', '幫', '你', '護航', '，', '而', '不', '讓', '你', '來', '專案', '報告', '。', '這個', '部分', '我要', '跟', '高雄市', '民', '澄清', '，', '我們', '高銀', '也', '非常', '願意來', '議會作', '專案', '報告',

In [9]:
punctuation = " //，：:""()\n!！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
re_punctuation = "[{}] ".format(punctuation)

#creat a df to store the jieba result
ly_docs = pd.DataFrame(columns=['jieba_results']) # 建立df
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

jieba.set_dictionary("KCC Data/Dict/dict.big.txt") #更改預設辭典（dict.big為繁體版）
jieba.load_userdict("KCC Data/Dict/KccDict2020.txt") #領域辭典
jieba.analyse.set_stop_words("KCC Data/Dict/KccStopWord2020.txt")

stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords: #斷詞辭典
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

for i in doc_names: #從第一篇到最後一篇
    words = jieba.cut(text_list[i], cut_all = False)
    text = ''
    for word in words:
        text = text + ' ' + str(word) 
    #print(words)
    #print(i) #第幾篇文章
    #print(text) #第幾篇文章的斷詞結果
    text = re.sub(re_punctuation, "", text) # 把上面那些符號從text中刪掉
    text = re.sub(r'[0-9]','',text) # 刪掉數字
    text = re.sub(r'[a-zA-Z]','',text) #刪掉英文字
#刪除文字 更多標點符號
    text = re.sub(r'[-<>#。！⋯.➡?=&▶_%♀!❗🎉⏰💪㊙️✨🔺😁😅😍⬇🙏✅🎈🔥📱👍👊🔸📍👉📢👩📌💼]','',text)  # 注意裡面不能有空白！
    s = pd.Series({'jieba_results': text})
    ly_docs = ly_docs.append(s, ignore_index=True)#每次的斷詞結果都加回liu_docs

Building prefix dict from C:\Users\User\OneDrive\POLITICS-AND-INFORMATION-master\20210517_code\KCC Data\Dict\dict.big.txt ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ue06d503e3529cb2586146b6160a5167a.cache
Loading model cost 1.205 seconds.
Prefix dict has been built successfully.


In [10]:
ly_docs

jieba_results
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [11]:
ly_docs.to_csv('國民黨團_output.txt')

In [12]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [13]:
fList = []
with open("國民黨團_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:1] = ", fList[:1])
print("------------------------------")

fList[:1] =  [['jieba_results']]
------------------------------


In [14]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(4856 unique tokens: ['jieba_results', '一下', '一切都是', '一堂', '一定']...)
------------------------------


In [15]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [16]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("國民黨團_ldaoutput.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.011*"我們" + 0.010*"市政府" + 0.010*"支持" + 0.008*"他們" + 0.008*"希望" + 0.007*"主席" + 0.007*"康議長裕成" + 0.007*"溝通" + 0.006*"能夠" + 0.006*"陳情" + 0.006*"反對" + 0.006*"民進黨團" + 0.005*"高雄" + 0.005*"時間" + 0.005*"謝謝" + 0.005*"建設" + 0.005*"過程" + 0.005*"聲音" + 0.005*"有些" + 0.005*"公開"

strTM =  10.016*"市長" + 0.016*"我們" + 0.015*"你們" + 0.013*"高雄市" + 0.012*"高雄" + 0.010*"許代理市長立明" + 0.009*"現在" + 0.009*"一個" + 0.008*"沒有" + 0.008*"許議員崑源" + 0.008*"回答" + 0.008*"代理" + 0.007*"輕軌" + 0.006*"看到" + 0.006*"什麼" + 0.006*"但是" + 0.005*"不要" + 0.005*"知道" + 0.005*"市民" + 0.005*"一定"

strTM =  20.000*"我們" + 0.000*"市長" + 0.000*"沒有" + 0.000*"你們" + 0.000*"高雄市" + 0.000*"陳議員美雅" + 0.000*"一個" + 0.000*"現在" + 0.000*"如果" + 0.000*"主席" + 0.000*"問題" + 0.000*"議員" + 0.000*"學校" + 0.000*"這樣" + 0.000*"高雄" + 0.000*"但是" + 0.000*"他們" + 0.000*"陳市長菊" + 0.000*"時間" + 0.000*"不要"

strTM =  30.027*"市長" + 0.026*"我們" + 0.018*"高雄市" + 0.012*"高雄" + 0.010*"建設" + 0.010*"但是" + 0.007*"看到" + 0.007*"市民" + 0.006*"沒有" + 0.006*"可以" + 0.006*"所有" + 0.005*"現在" + 0.005

In [17]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)